# CONSERVATION AND VARIATION ANALYSIS

Included in this notebook are the plots found in the manuscript as well as the code used to perform the analysis that led to the conclusions presented in the same manuscript.

### IMPORTING LIBRARIES

In [1]:
# NECESSARY IMPORTS TO RUN THE NOTEBOOK
import variant_analysis
import structural_analysis
import pandas as pd
import numpy as np
import importlib

### READING INPUT DATA

In [2]:
aln_in = "/cluster/gjb_lab/2394007/varalign_runs/varalign_edited_code2/anks_final_aln.sto" # FILEPATH TO INPUT ALIGNMENT
aln_fmt = "stockholm" # INPUT ALIGNMENT FORMAT
variant_table = pd.read_pickle("/cluster/gjb_lab/2394007/varalign_runs/varalign_edited_code2/.varalign/aligned_variants_data/anks_final_aln.sto_variants.p.gz") # VARIANT TABLE

## CONSERVATION ANALYSIS

In [7]:
cons_cols = structural_analysis.get_cons_cols(aln_in, aln_fmt, t = 0.45) # CONTAINS INDICES OF MSA COLUMNS WITH OCCUPANCY > 0.45 OR CONSENSUS
shenkin_aln = variant_analysis.calculate_shenkin(aln_in, aln_fmt)
shenkin_aln_filt = shenkin_aln[shenkin_aln.occ_pct > 0.45]
shenkin_aln_filt.index = range(1, len(shenkin_aln_filt)+1) # CONTAINS SHENKIN SCORE, OCCUPANCY/GAP PROPORTION OF CONSENSUS COLUMNS

In [14]:
min_shenkin = min(shenkin_aln_filt.shenkin)
max_shenkin = max(shenkin_aln_filt.shenkin)
shenkin_aln_filt["norm_shenkin_rel"] = 100*(shenkin_aln_filt.shenkin-min_shenkin)/(max_shenkin-min_shenkin) # ADDING NEW COLUMNS WITH DIFFERENT NORMALISED SCORES
shenkin_aln_filt["norm_shenkin_abs"] = 100*(shenkin_aln_filt.shenkin-6)/(120-6)

/cluster/gjb_lab/2394007/miniconda/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/cluster/gjb_lab/2394007/miniconda/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [15]:
shenkin_aln_filt.head()

,col,shenkin,occ,gaps,occ_pct,gaps_pct,norm_shenkin_rel,norm_shenkin_abs
1,38,92.851497,7173,234,0.968408,0.031592,87.448120,76.185524
2,39,39.034573,7196,211,0.971513,0.028487,26.661100,28.977696
3,40,103.964125,7233,174,0.976509,0.023491,100.000000,85.933443
4,41,33.590378,7309,98,0.986769,0.013231,20.511800,24.202086
5,42,33.560057,7335,72,0.990279,0.009721,20.477552,24.175488


In [16]:
variant_analysis.plot_conservation_mod(shenkin_aln_filt, colors = ["steelblue", "forestgreen", "gold", "firebrick"],
                                       cons_score_col = 'norm_shenkin_rel', cons_score_label = "Normalised Shenkin divergence score", qs = [25, 50, 75]) # PLOT NORMALISED SHENKIN DIVERGENCE SCORE PER POSITION

## MISSENSE VARIANTS ANALYSIS

In [17]:
human_missense_variants = variant_analysis.format_variant_table(variant_table, cons_cols) # GET ONLY MISSENSE VARIANTS ROWS

In [33]:
importlib.reload(variant_analysis)

<module 'variant_analysis' from '/cluster/home/2394007/variant_analysis.py'>

In [28]:
aln_out_human_missense_variants = "/cluster/gjb_lab/2394007/varalign_runs/varalign_edited_code2/anks_final_aln_human_missense_variants_seqs.sto"
missense_variants_df = variant_analysis.get_missense_df(
    aln_in, aln_fmt, human_missense_variants, cons_cols,
    shenkin_aln_filt, aln_out_human_missense_variants, t = 0.45
) # CALCULATES ENRICHMENT IN MISSENSE VARIANTS AND ASSOCIATED P-VALUE AND 95% CI

In [21]:
missense_variants_df.head()

,col,shenkin,occ,gaps,occ_pct,gaps_pct,norm_shenkin_rel,norm_shenkin_abs,variants,oddsratio,log_oddsratio,pvalue,ci_dist
1,38,92.851497,7173,234,0.968408,0.031592,87.448120,76.185524,628,0.958508,-0.042377,0.328772,0.082811
2,39,39.034573,7196,211,0.971513,0.028487,26.661100,28.977696,668,1.018187,0.018023,0.663817,0.080560
3,40,103.964125,7233,174,0.976509,0.023491,100.000000,85.933443,588,0.888038,-0.118741,0.006146,0.085258
4,41,33.590378,7309,98,0.986769,0.013231,20.511800,24.202086,704,1.057801,0.056192,0.163100,0.078664
5,42,33.560057,7335,72,0.990279,0.009721,20.477552,24.175488,752,1.128444,0.120840,0.002189,0.076409


In [31]:
missense_variants_df = variant_analysis.add_miss_class(missense_variants_df, cons_col = "norm_shenkin_rel", thresholds = [25,75]) # ADDS CLASSIFICATION ACCORDING TO MES AND NORMALISED SHENKIN

In [34]:
missense_variants_df.head()

,col,shenkin,occ,gaps,occ_pct,gaps_pct,variants,norm_shenkin_rel,norm_shenkin_abs,oddsratio,log_oddsratio,pvalue,ci_dist,miss_class,miss_color
1,38,92.851497,1401,32,0.977669,0.022331,628,87.448120,76.185524,0.962062,-0.038676,0.438413,0.095574,UMD,firebrick
2,39,39.034573,1404,29,0.979763,0.020237,668,26.661100,28.977696,1.023084,0.022821,0.631683,0.093609,None,grey
3,40,103.964125,1410,23,0.983950,0.016050,588,100.000000,85.933443,0.893106,-0.113050,0.023259,0.097636,UMD,firebrick
4,41,33.590378,1426,7,0.995115,0.004885,704,20.511800,24.202086,1.062927,0.061026,0.193316,0.091797,CME,green
5,42,33.560057,1430,3,0.997906,0.002094,752,20.477552,24.175488,1.134792,0.126449,0.006218,0.089841,CME,green


In [32]:
variant_analysis.plot_shenkin_logOR(
    missense_variants_df, "miss_class", "miss_color",
    markers = ["D", "s", "o", "h", "^"],
    colors = ["royalblue", "green", "grey", "firebrick", "orange"],
    cons_col = "norm_shenkin_rel", thresholds = [25,75])